In [1]:
import os
import json
import pandas as pd
import traceback

In [8]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
KEY=os.getenv("OPENAI_API_KEY")

In [13]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain_community.callbacks.manager import get_openai_callback
import PyPDF2

In [23]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }
}


In [31]:
# Creating template
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text,it is your job to\
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""


In [32]:
quiz_gen_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [33]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key="quiz",verbose=True)

In [35]:
# Another Template for checking the answer is correct or not
TEMPLATE2="""
You are an expert english grammarian and writer.Given a Multiple choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.\
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}
"""

In [36]:
quiz_eval_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [37]:
review_chain=LLMChain(llm=llm,prompt=quiz_eval_prompt,output_key="review",verbose=True)

In [38]:
generate_eval_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                    output_variables=["quiz","review"],verbose=True)

In [39]:
# Created a text file regarding a subject(Ex:Machine learning which is copies from wikipedia).
# To read that file we have to give the file path.
file_path=r"C:\Users\DELL\mcqgenr\data.txt"


In [40]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [42]:
# For converting/Serialize the RESPONSE from dictionary to json which is given in RESPONSE_JSON variable to json formatted string.

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [44]:
# To initialise the variables which will be described in callback_api func
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [45]:
# Initialising the getopenapi_callback which is used to see all the input,output prompts and also usage cost.
with get_openai_callback() as cb:
    response=generate_eval_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of 

In [46]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")






Total tokens: 1971
Prompt tokens: 1592
Completion Tokens: 379
Total Cost: 0.0031459999999999995


In [47]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [49]:
quiz=response.get("quiz")

In [52]:
quiz=json.loads(quiz)

In [55]:
import pandas as pd

# Assuming `quiz` is your dictionary containing quiz data
quiz_data = []
for key, value in quiz.items():
    mcq = value['mcq']  # Assuming 'mcq' is the key for the MCQ question
    options = " | ".join([f"{option}:{option_value}" for option, option_value in value['options'].items()])
    correct = value['correct']  # Assuming 'correct' is the key for the correct answer

    quiz_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

# Convert to DataFrame
df = pd.DataFrame(quiz_data)

# Display the DataFrame (optional)
print(df)


                                                 MCQ  \
0                          What is machine learning?   
1  Which field does machine learning find applica...   
2              Who coined the term machine learning?   
3  What was the earliest machine learning model i...   
4  What is the main objective of modern-day machi...   

                                             Choices Correct  
0  a:A field of study in artificial intelligence ...       a  
1  a:Agriculture and music | b:Natural language p...       b  
2  a:Elon Musk | b:Arthur Samuel | c:Bill Gates |...       b  
3  a:Weather prediction model | b:Checkers winnin...       b  
4  a:To confuse users with complex algorithms | b...       b  


In [56]:
df.to_csv("ML-quiz")